<a href="https://colab.research.google.com/github/Requenamar3/DeepLearning/blob/main/Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
df = pd.read_csv('https://raw.githubusercontent.com/fenago/datasets/main/winequalityN.csv')
df.sample(5)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3166,white,5.2,0.34,0.37,6.2,0.031,42.0,133.0,0.99076,3.25,0.41,12.5,6
804,white,7.3,0.32,0.34,6.6,0.032,24.0,112.0,0.99505,3.22,0.46,9.8,6
912,white,6.3,0.34,0.19,5.8,0.041,22.0,145.0,0.99430,3.15,0.63,9.9,5
3398,white,6.1,0.33,0.32,7.8,0.052,52.0,183.0,0.99657,3.39,0.65,9.5,5
335,white,6.3,0.23,0.30,1.8,0.033,16.0,91.0,0.99060,3.28,0.40,11.8,6


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare the data
df = df.dropna()
df['is_white_wine'] = [1 if typ == 'white' else 0 for typ in df['type']]
df['is_good_wine'] = [1 if quality >= 6 else 0 for quality in df['quality']]
df.drop(['type', 'quality'], axis=1, inplace=True)

# Train/test split
X = df.drop('is_good_wine', axis=1)
y = df['is_good_wine']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Tune the Learning Rate...
# Tune the network architecture
import tensorflow as tf

In [ ]:
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/model-{epoch:02d}-{val_accuracy:.2f}.hdf5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

In [ ]:
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.001,
    patience=10,
    verbose=1
)

In [ ]:
cb_csvlogger = tf.keras.callbacks.CSVLogger(
    filename='training_log.csv',
    separator=',',
    append=False
)

In [ ]:
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    factor=0.1,
    patience=10,
    verbose=1,
    min_lr=0.00001
)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

model.fit(
    X_train_scaled,
    y_train,
    epochs=10000,
    validation_data=(X_test_scaled, y_test),
    callbacks=[cb_checkpoint, cb_reducelr, cb_earlystop, cb_csvlogger]
)

Epoch 1/10000
156/162 [===========================>..] - ETA: 0s - loss: 0.5551 - accuracy: 0.7123
Epoch 1: val_accuracy improved from -inf to 0.76102, saving model to checkpoints/model-01-0.76.hdf5
162/162 [==============================] - 3s 5ms/step - loss: 0.5532 - accuracy: 0.7139 - val_loss: 0.4919 - val_accuracy: 0.7610 - lr: 0.0010
Epoch 2/10000
153/162 [===========================>..] - ETA: 0s - loss: 0.4995 - accuracy: 0.7569
Epoch 2: val_accuracy did not improve from 0.76102
162/162 [==============================] - 0s 3ms/step - loss: 0.4980 - accuracy: 0.7584 - val_loss: 0.4874 - val_accuracy: 0.7587 - lr: 0.0010
Epoch 3/10000
150/162 [==========================>...] - ETA: 0s - loss: 0.4825 - accuracy: 0.7731
Epoch 3: val_accuracy did not improve from 0.76102
162/162 [==============================] - 1s 4ms/step - loss: 0.4827 - accuracy: 0.7727 - val_loss: 0.4779 - val_accuracy: 0.7602 - lr: 0.0010
Epoch 4/10000
153/162 [===========================>..] - ETA: 0s - lo

In [ ]:
best_model = tf.keras.models.load_model('/content/checkpoints/model-30-0.79.hdf5')